In [ ]:
# 20231026回家作業說明
# 到環境部抓取aqx_p_07空氣品質監測站基本資料
# 參考網址 https://data.moenv.gov.tw/swagger/#/%E5%A4%A7%E6%B0%A3/get_aqx_p_07
# 建立password.py,提供api key,
# 建立連結抓取檔案（要確認csv檔或是json檔),
# 建立資料庫存取抓下來的資料
# 建立執行緒，進行每小時抓取一次資料